In [1]:
import os
import sys

# Get the absolute path of the scripts directory
root_path_scripts = os.path.abspath(os.path.join(os.getcwd(), '../scripts/'))

# Add the scripts directory to sys.path
sys.path.append(root_path_scripts)

# Import the function from load-docs.py
from load_docs import load_document 

In [2]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
# from langchain.embeddings import SentenceTransformerEmbeddings
# from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings

import openai
from openai import OpenAI

# load env variable
load_dotenv()

openai_client = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')

/home/air/Docker-day[30-day]/venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
# Load the document(s)
file_path = 'Robinson Advisory.docx'  # Replace with your actual file path
documents = load_document("../data/DerejeHinsermuSenbatu-resume.pdf")

In [4]:

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# Initialize Embedding model
# embedding_function = SentenceTransformerEmbeddingFunction()
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/ipykernel_11790/220872094.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/home/air/Docker-day[30-day]/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/air/Docker-day[30-day]/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted i

In [5]:

# Create Chroma Vector Store
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

# Perform Similarity Search
query = "Can you provide suggestions on how I can enhance my work experience"
retrieved_documents = vectorstore.similarity_search(query, k=5)

for doc in retrieved_documents:
    print(doc.page_content)
    print('\n')


improvement
in
client
satisfaction
scoring
an
average
of
90%.
Machine
Learning
Engineer:
Upwork,
Addis
Ababa,
Ethiopia,
Freelancer
December
2020
-
present
●
Created
customized
machine
learning
models
based
on
specific
client
requirements,
significantly
enhancing
prediction
accuracy
by
35%
through
iterative
feedback
and
refinement.
●
Conducted
precise
manual
data
annotation
to
improve
model
training
data,
which
resulted
in
a
20%
increase
in
overall
model
accuracy
and
effectiveness.
●
Engaged
with
clients
to
communicate
technical
concepts
and
results
effectively,
receiving
positive
feedback
on
project
delivery
and
stakeholder
satisfaction.
Junior
Generative
AI
Engineer
:
10
Academy,
California,
USA,
Remote
April
2024
-
July
2024
●
Developed
a
high-precision
Contract
Advisor
RAG
system
for
intricate
legal
contract
Q&A,
augmenting
efficiency
and
accuracy
in
contract
review
processes.
●
Employing
advanced
techniques
like
Query
Expansion
enhanced
the
performance
of
the
AI
system,
offering




In [8]:
def rag(query, retrieved_documents, model="gpt-4o-mini"):
    information = "\n\n".join([doc.page_content for doc in retrieved_documents])

    messages = [
        {
            "role": "system",
            "content": """
            You are an expert resume reviewer. Your role is to provide short and concise constructive feedback on how to improve resumes in alignment with industry standards.put your point in the form of bullet and add a new line after first point. Please follow these guidelines:

                Resume Length: Assess if my resume is concise and within 1 to 2 pages. If it exceeds this length, please suggest specific sections or content that can be condensed or removed while maintaining impact.

    Professional Summary: Evaluate my professional summary to ensure it's 50 words or less and effectively highlights my key skills and experience. Provide suggestions for improvement if necessary.

    Work Experience: Review each role in my work experience section using the XYZ model:
        X (What I did): Are my responsibilities clearly stated?
        Y (How I did it): Have I explained the methods or tools I used?
        Z (Impact): Do I highlight the outcomes or results of my actions? Please point out any areas lacking in this model and suggest enhancements.

    Education: Check if I've highlighted 3-5 relevant courses that align with the job I'm applying for. Advise on any additional courses to include or unnecessary ones to remove.

    Language and Tone: Identify instances of passive or vague language. Provide suggestions to replace them with dynamic and specific wording. Highlight any repetitive phrases that could be revised.

    Skills Section: Examine if I've focused on 8 to 12 core competencies relevant to my career goals and the job requirements. Suggest any additional hard skills that should be included or soft skills that could be de-emphasized.

    Project Links: Confirm whether I've included direct links to my projects (e.g., GitHub repositories, personal websites). If any are missing, recommend where they should be added.

    Portfolio Link: Verify if a link to my portfolio is present. If not, suggest how and where to incorporate it for maximum impact.

    Project Descriptions: Review my project descriptions to ensure they outline:
        The problem solved
        The approach taken
        The outcome achieved (with quantifiable results, if possible) Provide specific suggestions to enhance any descriptions that lack these elements.

    Relevance and Context: Assess whether the projects and experiences I've included are most relevant to the job I'm applying for. Advise on any that could be removed or any additional experiences that should be highlighted to showcase my skills and competencies better.

Please reference specific sections of my resume in your feedback and provide concrete suggestions for improvement."
            If there are no changes needed for a section, simply state that it is well-presented.If you don't know the answer, just say that you don't know, don't try to make up an answer
                        """
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [9]:
Answer = rag(query=query, retrieved_documents=retrieved_documents)

print(Answer)

### Work Experience Enhancements:

#### General Suggestions:
- **Formatting Consistency**: Ensure your bullet points are consistently formatted and properly spaced. This helps maintain clarity and readability.
- **Concise Descriptions**: Some descriptions can be more concise by removing redundant words or phrases while retaining critical information.

#### Specific Role Enhancements:

##### Machine Learning Engineer (Upwork, Addis Ababa, Ethiopia, Freelancer, December 2020 - Present)
- **Outcome and Impact**: Further detail the benefits to clients and end-users. Include metrics or KPIs where possible.
  - **Example**: Instead of "Created customized machine learning models based on specific client requirements, significantly enhancing prediction accuracy by 35% through iterative feedback and refinement," consider "Developed customized ML models tailored to clients' needs, improving prediction accuracy by 35%, which led to a 20% reduction in operational costs for clients."

##### Junior 